In [ ]:
import os
import json
import numpy as np

from people import datasetpath
from people.datasets.human36m import ZBOUND
from people.datasets.human36m import BBOX_REF

from people.utils import *

hashid0 = '34c85c' #'f4c43b'
hashid1 = '981233' #'1e8fcd'

inputdir = 'h36m_test_predictions_12-03-2019'

pred0_file = 'output/%s/h36m_test_pred_pose_hf0_%s.json' %(inputdir, hashid0)
pred1_file = 'output/%s/h36m_test_pred_pose_hf1_%s.json' %(inputdir, hashid1)
absz0_file = 'output/%s/h36m_test_pred_absz_hf0_%s.json' % (inputdir, hashid0)
absz1_file = 'output/%s/h36m_test_pred_absz_hf1_%s.json' % (inputdir, hashid1)
afmat0_file = 'output/%s/h36m_test_afmat_%s.json' % (inputdir, hashid0)
afmat1_file = 'output/%s/h36m_test_afmat_%s.json' % (inputdir, hashid1)
scam0_file = 'output/%s/h36m_test_scam_%s.json' % (inputdir, hashid0)
scam1_file = 'output/%s/h36m_test_scam_%s.json' % (inputdir, hashid1)


outdir = 'output/h36m_test_predictions_21-03-2019'
mkdir(outdir)
# mkdir(os.path.join(outdir, 'S2'))
# mkdir(os.path.join(outdir, 'S3'))
# mkdir(os.path.join(outdir, 'S4'))

crop_res = (256, 256)
anchors_size = [(32, 32)]
h36m_dataconf = DataConfig(
        crop_resolution=crop_res,
        )
# h36m = Human36MTest(datasetpath('Human3.6M'), h36m_dataconf, anchors_size,
        # image_div=8, preprocess_mode='caffe')

"""Loading meta ids from h36m."""
with open(os.path.join(outdir, 'h36m_meta_test.json'), 'r') as fid:
    meta_h36m = json.load(fid)

with open(pred0_file, 'r') as fid:
    pred0 = np.array(json.load(fid))
with open(pred1_file, 'r') as fid:
    pred1 = np.array(json.load(fid))
pred1 = pred1[:, pa17j3d.map_hflip, :]

with open(absz0_file, 'r') as fid:
    z_pred0 = np.array(json.load(fid))
with open(absz1_file, 'r') as fid:
    z_pred1 = np.array(json.load(fid))

with open(afmat0_file, 'r') as fid:
    afmat0 = np.array(json.load(fid))
with open(afmat1_file, 'r') as fid:
    afmat1 = np.array(json.load(fid))

with open(scam0_file, 'r') as fid:
    scam0 = np.array(json.load(fid))
with open(scam1_file, 'r') as fid:
    scam1 = np.array(json.load(fid))

"""Project normalized coordiates to the image plane."""
pred0[:, :, 0:2] = transform_pose_sequence(
        afmat0, pred0[:, :, 0:2], inverse=True)
pred1[:, :, 0:2] = transform_pose_sequence(
        afmat1, pred1[:, :, 0:2], inverse=True)

"""Recover the absolute Z."""
z_pred0 = z_pred0 * (ZBOUND[1] - ZBOUND[0]) + ZBOUND[0]
z_pred1 = z_pred1 * (ZBOUND[1] - ZBOUND[0]) + ZBOUND[0]

pred0[:, :, 2] = (BBOX_REF * (pred0[:, :, 2] - 0.5)) \
        + np.expand_dims(z_pred0, axis=-1)
pred1[:, :, 2] = (BBOX_REF * (pred1[:, :, 2] - 0.5)) \
        + np.expand_dims(z_pred1, axis=-1)

"""Map to h36m layout of 17j 3D."""
"""The 17 joints - in order - are: {
    'Pelvis'    # 0
    'RHip'      # 10
    'RKnee'     # 12
    'RAnkle'    # 14
    'LHip'      # 11
    'LKnee'     # 13
    'LAnkle'    # 15
    'Spine1'    # 16
    'Neck'      # 1
    'Head'      # 2
    'Site'      # 3
    'LShoulder' # 5
    'LElbow'    # 7
    'LWrist'    # 9
    'RShoulder' # 4
    'RElbow'    # 6
    'RWrist     # 8
    }
"""
map_pa17j_to_h36m17j = np.array([
    0, 10, 12, 14, 11, 13, 15, 16, 1, 2, 3, 5, 7, 9, 4, 6, 8])
# map_h36m17j_to_pa17j = np.array([
    # 0, 8, 9, 10, 14, 11, 15, 12, 16, 13, 1, 4, 2, 5, 3, 6, 7])

pred0 = pred0[:, map_pa17j_to_h36m17j, :3]
pred1 = pred1[:, map_pa17j_to_h36m17j, :3]
puvd = (pred0 + pred1) / 2.

In [ ]:
cam_to_use = [1, 2, 3, 4]
pose_c_pred = np.zeros(puvd.shape)
alpha = 1 / len(cam_to_use)

i = 0
while i < 20*3050:
#while i < len(pose_c_pred):

    a, s, e, c, f = meta_h36m[i]
    print (a, s, e, c, f)

    idxs = {}
    for ic in range(4):
        idxs[ic+1] = get_idxs_sequence(meta_h36m, asec=(a, s, e, ic+1))

    try:
        pairlist = cam_to_use.copy()
        del pairlist[pairlist.index(c)]
    except:
        pass

    puvd1 = puvd[idxs[c]]
    if len(puvd1) == 0:
        print ('Warning! Sample %d %d %d %d empty!' % (a, s, e, c))
        continue
    num_samples, num_joints, dim = puvd1.shape
    pc_est = np.zeros((num_samples * num_joints, dim))

    for pc in pairlist:
        puvd2 = puvd[idxs[pc]]
        if len(puvd2) == 0:
            print ('Warning! Sample %d %d %d %d empty!' % (a, s, e, c))
            continue

        puvd1 = np.reshape(puvd1, (num_samples * num_joints, dim))
        puvd2 = np.reshape(puvd2, (num_samples * num_joints, dim))

        f1, c1, f2, c2, P21, P12, loss = predict_camera_parameters(puvd1, puvd2)
        #print (f1, c1, f2, c2)
        #plt.plot(loss)

        pc1_est = predict_xy_mm(puvd1, f1, c1)
        pc2_est = predict_xy_mm(puvd2, f2, c2)
        pc_est += alpha * pc1_est + (1 - alpha) * project_poses(pc2_est, P21)

    pc_est /= len(pairlist)
    #pc_gt = pose_c_gt[idxs[c]]
    #print (abs_mpjpe(pc_est, pc_gt), rel_mpjpe(pc_est, pc_gt))

    pc_est = np.reshape(pc_est, (num_samples, num_joints, dim))
    pose_c_pred[i:i+len(pc_est), :, :] = pc_est
    i += len(pc_est)

In [ ]:
pose_c_pred = pose_c_pred[:i]
pose_c_pred[:, :, :] -= pose_c_pred[:, 0:1, :]
prev_pred_file = 'output/h36m_test_predictions_25-02-2019/h36m_test_pred_c_f4c43b_multicamera.json'
with open(prev_pred_file, 'r') as fid:
    prev_pred = json.load(fid)
prev_pred = np.array(prev_pred)[:i]
abserr = abs_mpjpe(prev_pred, pose_c_pred)
relerr = rel_mpjpe(prev_pred, pose_c_pred)
print (abserr, relerr)

In [ ]:
pred_c_file = '%s/h36m_test_pred_absc_%s_multicamera.json' % (outdir, hashid0)
with open(pred_c_file, 'w') as fid:
    json.dump(pose_c_pred.tolist(), fid)

In [ ]:
pose_c_pred[:, :, :] -= pose_c_pred[:, 0:1, :]
pred_c_file = '%s/h36m_test_pred_c_%s_multicamera.json' % (outdir, hashid0)
with open(pred_c_file, 'w') as fid:
    json.dump(pose_c_pred.tolist(), fid)

In [ ]:
prev_pred_file = 'output/h36m_test_predictions_25-02-2019/h36m_test_pred_c_f4c43b_multicamera.json'
with open(prev_pred_file, 'r') as fid:
    prev_pred = json.load(fid)
prev_pred = np.array(prev_pred)

In [ ]:
abserr = abs_mpjpe(prev_pred, pose_c_pred)
relerr = rel_mpjpe(prev_pred, pose_c_pred)
print (abserr, relerr)

In [ ]:
import os
import json
import numpy as np

from people import datasetpath
from people.datasets.human36m import ZBOUND
from people.datasets.human36m import BBOX_REF

from people.utils import *

with open('output/h36m_test_predictions_20-03-2019/h36m_test_pred_c_f4c43b_multicamera.json', 'r') as fid:
    cur_pred = np.array(json.load(fid))
with open('output/h36m_test_predictions_25-02-2019/h36m_test_pred_c_f4c43b_multicamera.json', 'r') as fid:
    prev_pred = np.array(json.load(fid))
print (abs_mpjpe(cur_pred, prev_pred), rel_mpjpe(cur_pred, prev_pred))